<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/Modeltf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install joblib
import tensorflow as tf 
import pickle
#from sklearn.externals import joblib
import numpy as np
from tensorflow.python.client import device_lib
import random
import matplotlib.pyplot as plt 
import math
import warnings
from sklearn.model_selection import train_test_split

In [36]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
## 64 BIT (2253,4016)
'''
with open('/content/drive/My Drive/ffeatures64.bin','rb') as file:
  features = pickle.load(file)
with open('/content/drive/My Drive/flabels64.bin','rb') as file:
  labels = pickle.load(file)
'''

In [0]:
features = joblib.load('')
labels = joblib.load('')

In [0]:
# print(features[0])
# print(len(features[0]))
# print(len(features))
print(np.shape(features))

(2253, 4106)


In [0]:
## TEST SET 

x_train_t, x_test_t,y_train_t, y_test_t = train_test_split(features, labels, test_size = 0.3)

In [0]:
print(np.shape(x_train_t))

(1577, 4106)


In [0]:
tf.reset_default_graph()

In [0]:
print(device_lib.list_local_devices())


In [0]:
x = tf.placeholder(shape = (None,266), dtype = tf.float32, name = 'input' )
y = tf.placeholder(shape = (None,1), dtype = tf.float32, name = 'output')

wh = tf.Variable(tf.glorot_uniform_initializer(dtype = tf.float32)((266, 500)))
bh = tf.Variable(tf.glorot_uniform_initializer(dtype = tf.float32)((500,1)))

#out

wo = tf.Variable(tf.glorot_uniform_initializer(dtype = tf.float32)((500,1)))
bo = tf.Variable(tf.glorot_uniform_initializer(dtype = tf.float32)((1,1)))


                

In [0]:
hidl = tf.add(tf.matmul(x,wh),bh)

In [0]:
outl = tf.add(tf.matmul(hidl,wo),bo)

Add additional custom loss functions if necc 

In [0]:
loss = tf.reduce_mean(tf.square(tf.subtract(y,outl)))
#default lr = 0.001
opt = tf.train.AdamOptimizer(name = 'optimizer')
op_train = opt.minimize(loss, name ='train_op')

In [0]:
saved = tf.train.Saver().as_saver_def()

In [0]:
init = tf.global_variables_initializer()

In [0]:
with tf.device('/device:CPU:0'):
  save = tf.train.Saver()
  with tf.Session() as sess:
    sess.run(init)
    mini =[]
    for i in range(0,100):
      b1=[]
      b2=[]
      l1=[]
      l2=[]
      x_trainv, x_testv, y_trainv, y_testv = train_test_split(x_train_t, y_train_t, test_size = 0.3)
      for j in range(16): ##One run is appending 2 random lists 
        seed = random.choice(range(len(x_trainv)))
        i = random.choice(range(len(x_trainv)))
        b1.append(x_trainv[seed])
        b2.append(x_trainv[i])
        l1.append(y_trainv[seed])
        l2.append(y_trainv[i])
        x_batch = np.vstack((b1,b2))
        y_batch = np.vstack((l1,l2))
        
        
        
    sess.run([loss,op_train], feed_dict = {x : x_batch, y: y_batch })
    m_save(sess,'/content/drive/My Drive/Custom_Models/model1',inputs = {'input' : x}, outputs = {'outputs': out})
    print('Saved model!')
    

In [0]:
batch = np.asarray(mini,dtype = np.float32)

In [0]:
print(len(batch))

32


In [0]:
print(np.shape(mini))
print(np.ndarray.flatten(mini[0]))
print(mini[0])

In [0]:
tf.trainable_variables()